<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Wav2Vec2%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install the below libraries required to run this notebook

In [ ]:
# Import the necessary libraries
!pip install datasets
!pip install transformers
!pip install torch
!pip install evaluate
!pip install jiwer
!pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.4 MB/s eta 0:00:00

In [ ]:
##Imports required
import numpy as np
from datasets import load_dataset, Audio
from transformers import MCTCTProcessor, MCTCTForCTC
import torch
import evaluate
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import TrainingArguments, Trainer

## LOADING AND INFERRING FROM THE BASE MODEL

In [ ]:
# Load the PolyAI dataset.
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train[:80]")

# Remove unnecessary columns
dataset = dataset.remove_columns(['path','english_transcription','intent_class'])

# Split the dataset into train and test
dataset = dataset.train_test_split(test_size = 0.2, shuffle=False)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Declare device variable
device = 'cuda' if torch.cuda.is_available() else'cpu'

## Resample the dataset to 16 Khz as MCTCT model is trained on 16khz
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
# Loading the model and autoprocessor  from transformer
# Autoprocessor is a wrapper of feature extractor and tokenizer

processor = MCTCTProcessor.from_pretrained("speechbrain/m-ctc-t-large")
model = MCTCTForCTC.from_pretrained("speechbrain/m-ctc-t-large")
model.to(device)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MCTCTForCTC(
  (mctct): MCTCTModel(
    (encoder): MCTCTEncoder(
      (layer_norm): MCTCTLayerNorm()
      (conv): MCTCTConv1dSubsampler(
        (dropout): Dropout(p=0.3, inplace=False)
        (conv_layers): ModuleList(
          (0): Conv1d(80, 3072, kernel_size=(7,), stride=(3,), padding=valid)
        )
      )
      (layers): ModuleList(
        (0-35): 36 x MCTCTLayer(
          (intermediate): MCTCTIntermediate(
            (dense): Linear(in_features=1536, out_features=6144, bias=False)
            (intermediate_act_fn): ReLU()
          )
          (attention): MCTCTAttention(
            (self): MCTCTSelfAttention(
              (query): Linear(in_features=1536, out_features=1536, bias=False)
              (key): Linear(in_features=1536, out_features=1536, bias=False)
              (value): Linear(in_features=1536, out_features=1536, bias=False)
              (dropout): Dropout(p=0.3, inplace=False)
              (distance_embedding): Embedding(1839, 384)
            )
    

In [ ]:
# Lets process the first example of train dataset

inputs = processor(dataset['train'][3]["audio"]["array"], sampling_rate=16000, return_tensors="pt")

In [ ]:
# Lets try to run inference on one of the train example


with torch.no_grad():
    logits = model(**inputs.to(device)).logits


predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
transcription

['How do wy started to an account?']

In [ ]:
# Checking with actual transcription in the dataset

dataset['train'][3]['transcription']

'how do I start a joint account'

### FINE TUNING THE MODEL

In [ ]:
# Preparing a function to process the entire dataset
# We need to crate two variables with name 'input_featrues'(input array of sound wave in raw foram) and 'labels'(transcription)

def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_features"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [ ]:
encoded_dataset = dataset.map(prepare_dataset, num_proc=4)

Map (num_proc=4):   0%|          | 0/64 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/deprecated/mctct/processing_mctct.py:132: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/deprecated/mctct/processing_mctct.py:132: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/deprecated/mctct/processing_mctct.py:132: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the 

Map (num_proc=4):   0%|          | 0/16 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/deprecated/mctct/processing_mctct.py:132: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/deprecated/mctct/processing_mctct.py:132: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/deprecated/mctct/processing_mctct.py:132: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the 

In [ ]:
#encoded_dataset["train"] = encoded_dataset["train"].filter(lambda x: len(x) < 1500, input_columns=["input_features"])

In [ ]:
# Creating a DataCollatorClass

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: MCTCTProcessor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")

In [ ]:

# Evalution metric- We will be evaluating our model on word error rate



wer = evaluate.load("wer")

In [ ]:


def compute_metrics(pred):
    wer = evaluate.load("wer")
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
del model
model = MCTCTForCTC.from_pretrained('speechbrain/m-ctc-t-large',ctc_loss_reduction="mean",pad_token_id=processor.tokenizer.pad_token_id)
model.to(device)

MCTCTForCTC(
  (mctct): MCTCTModel(
    (encoder): MCTCTEncoder(
      (layer_norm): MCTCTLayerNorm()
      (conv): MCTCTConv1dSubsampler(
        (dropout): Dropout(p=0.3, inplace=False)
        (conv_layers): ModuleList(
          (0): Conv1d(80, 3072, kernel_size=(7,), stride=(3,), padding=valid)
        )
      )
      (layers): ModuleList(
        (0-35): 36 x MCTCTLayer(
          (intermediate): MCTCTIntermediate(
            (dense): Linear(in_features=1536, out_features=6144, bias=False)
            (intermediate_act_fn): ReLU()
          )
          (attention): MCTCTAttention(
            (self): MCTCTSelfAttention(
              (query): Linear(in_features=1536, out_features=1536, bias=False)
              (key): Linear(in_features=1536, out_features=1536, bias=False)
              (value): Linear(in_features=1536, out_features=1536, bias=False)
              (dropout): Dropout(p=0.3, inplace=False)
              (distance_embedding): Embedding(1839, 384)
            )
    

In [ ]:
# defining training arguments and trainer

training_args = TrainingArguments(
    output_dir="m-ctc-t_trained",
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    learning_rate=1e-5,
    warmup_steps=2,
    max_steps=2000,
    fp16=True,
    optim='adafactor',
    group_by_length=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=1,
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="wer",

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



In [ ]:
# Requires GPU for training
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,No log,1.023276,0.467742
200,No log,1.218436,0.435484
300,No log,1.160553,0.443548
400,No log,1.444840,0.435484
500,3.066700,1.962108,0.403226
600,3.066700,2.487072,0.403226
700,3.066700,2.188133,0.435484
800,3.066700,2.187747,0.459677
900,3.066700,2.157700,0.419355
1000,1.222900,2.505741,0.403226


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=2000, training_loss=1.399481414794922, metrics={'train_runtime': 1857.3649, 'train_samples_per_second': 1.077, 'train_steps_per_second': 1.077, 'total_flos': 6.808141546555752e+17, 'train_loss': 1.399481414794922, 'epoch': 31.25})

In [ ]:
with torch.no_grad():
    logits = model(**inputs.to(device)).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
transcription

['how do I start a joint account']

In [ ]:
## getting test data
i2 = processor(dataset['test'][6]["audio"]["array"], sampling_rate=16000, return_tensors="pt")
print(f"The input test audio is: {dataset['test'][6]['transcription']}")

# prediction for test data
with torch.no_grad():
    logits = model(**i2.to(device)).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(f'The output prediction is : {transcription[0]}')

The input test audio is: so you spent the money I'd like to see my new account balance
The output prediction is : so I just spent some money I'd like to see my new account balance
